In [5]:
import os
import sys

# 获取当前工作目录
current_dir = os.getcwd()
# 设置项目根目录
project_root = os.path.abspath(os.path.join(current_dir, ".."))
# 将项目根目录添加到sys.path
sys.path.insert(0, project_root)

In [6]:

from db import es
from api import deepseek
from agent import retrieval

In [ ]:
question = 'Director接收南向告警的snmp端口号是多少'

In [15]:
kws = deepseek.extract_keywords(question)
print(kws)
result = es.search_by_root_and_keywords('director', kws)['hits']['hits']
contents = [hit['_source'] for hit in result]
for content in contents:
    print(content['url'], content['keywords'], content['content'])
    print('===================================\n')

In [9]:
result = retrieval.retrieve(question, 'director')
print(result)

In [20]:
a = es.search_by_content('南向告警', 'director')['hits']['hits']
contents = [hit['_source'] for hit in a]
for content in contents:
    print(content.get('url'), content['keywords'], content['content'])
    print('===================================\n')

## 通过llm遍历文档树去找文件

In [71]:
from utils import node_util
from api import glm

prompt_tmp = """
你是一名技术专家，我有一些运维材料文档，文档名称如下：

{names}

我现在有一个问题：{question}

你觉得它最有可能在哪些文档下？请直接输出文档名称，若有多个，请用英文逗号隔开。

注意：
1. 如果你觉得这些文档里面都没有答案，或者你不确定，都直接返回无关两个字就行
2. 只返回逗号隔开的文档名称，禁止输出任何其他文字。
输出格式如下：

{{doc1}},{{doc2}},{{doc3}}
其中大括号里面表示的是文档名称
"""

def filter(nodes):
    names = [node.name for node in nodes]
    print(f"输入: {names}")
    prompt = prompt_tmp.format(names=names, question=question)
    result = glm.chat(prompt).replace('\'','')
    print(f"输出: {result}")
    arr = [item for item in result.split(',') if item != '无关']
    last = [node for node in nodes if node.name in arr]
    return last


def process_nodes(nodes):
    result = []
    while nodes:
        filtered_nodes = filter(nodes)
        result.extend(filtered_nodes)
        all_children = []
        for node in filtered_nodes:
            all_children.extend(node.children)
        if not all_children:
            break
        nodes = all_children
    return result


In [72]:
root_path = 'D:\\Workspace\\aiops2024-challenge-dataset\\director\\'
root_nodes = node_util.parse_xml(root_path + 'nodetree.xml')
result_nodes = process_nodes(root_nodes)
# for n in result_nodes:
#     print(n.name)

In [64]:
from pre import html_processor

# 按照url去读取文件
def read(name, url):
    dst_url = root_path + '\\documents\\' + url

    content = html_processor.html_to_markdown(dst_url)
    
    if len(content) > 1000:
        print(name)
    else:
        print(content)
        print('\n\n')
for node in result_nodes:
    read(node.name, node.url)